In [37]:
import pandas as pd
import os.path as osp

In [38]:
result_pth = './results_a2t_sst2'

In [40]:
log_df = pd.read_csv(osp.join(result_pth,'log.csv'))
out_df = log_df[['original_text', 'perturbed_text', 'result_type']]
out_df

,original_text,perturbed_text,result_type
0,it 's a charming and often affecting journey .,it 's a charming and normally affecting journe...,Failed
1,unflinchingly bleak and desperate,unflinchingly grim and desperate,Failed
2,allows us to hope that nolan is poised to emba...,authorizes us to hopes that nolan is prepped t...,Failed
3,"the acting , costumes , music , cinematography...","the acting , costumes , music , cinematography...",Successful
4,"it 's slow -- very , very slow .","it 's slow -- very , very slower .",Failed
...,...,...,...
867,has all the depth of a wading pool .,has all the depths of a wading pool .,Successful
868,a movie with a real anarchic flair .,a films with a true anarchic flair .,Failed
869,a subject like this should inspire reaction in...,a subject like this should inspiring reply in ...,Successful
870,... is an arthritic attempt at directing by ca...,... is an arthritic endeavour at directing by ...,Successful


In [41]:
log_df

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,it 's a charming and often affecting journey .,it 's a charming and normally affecting journe...,0.000234,0.000557,1.0,1.0,1.0,14.0,Failed
1,unflinchingly bleak and desperate,unflinchingly grim and desperate,0.014329,0.154515,0.0,0.0,0.0,5.0,Failed
2,allows us to hope that nolan is poised to emba...,authorizes us to hopes that nolan is prepped t...,0.000426,0.283926,1.0,1.0,1.0,32.0,Failed
3,"the acting , costumes , music , cinematography...","the acting , costumes , music , cinematography...",0.003972,0.622039,1.0,0.0,1.0,8.0,Successful
4,"it 's slow -- very , very slow .","it 's slow -- very , very slower .",0.002048,0.002501,0.0,0.0,0.0,4.0,Failed
...,...,...,...,...,...,...,...,...,...
867,has all the depth of a wading pool .,has all the depths of a wading pool .,0.185487,0.851692,0.0,1.0,0.0,2.0,Successful
868,a movie with a real anarchic flair .,a films with a true anarchic flair .,0.000322,0.000818,1.0,1.0,1.0,5.0,Failed
869,a subject like this should inspire reaction in...,a subject like this should inspiring reply in ...,0.136275,0.518427,0.0,1.0,0.0,13.0,Successful
870,... is an arthritic attempt at directing by ca...,... is an arthritic endeavour at directing by ...,0.003134,0.734410,0.0,1.0,0.0,11.0,Successful


In [3]:
transformation_log = pd.read_csv('../results/transformation.csv', index_col=0, names = ["transformation_id","transformation",
            "prev_text", "after_text", "prev_target", "after_target","from_modified_indices", "to_modified_indices", "changes"])
transformation_log

,transformation,prev_text,after_text,prev_target,after_target,from_modified_indices,to_modified_indices,changes
transformation_id,,,,,,,,
0,"{'class': 'WordSwapEmbedding', 'max_candidates...",0,1,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
1,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,3,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
2,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,4,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
3,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,5,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
4,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,6,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
...,...,...,...,...,...,...,...,...
2075,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2095,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"
2076,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2096,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"
2077,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2097,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"


In [4]:
edge_to_transformation = transformation_log.set_index(['prev_text', 'after_text'])
edge_to_transformation

transformation  \
prev_text after_text                                                      
0         1           {'class': 'WordSwapEmbedding', 'max_candidates...   
2         3           {'class': 'WordSwapEmbedding', 'max_candidates...   
          4           {'class': 'WordSwapEmbedding', 'max_candidates...   
          5           {'class': 'WordSwapEmbedding', 'max_candidates...   
          6           {'class': 'WordSwapEmbedding', 'max_candidates...   
...                                                                 ...   
1982      2095        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2096        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2097        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2098        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2099        {'class': 'WordSwapEmbedding', 'max_candidates...   

                      prev_target  after_target from_modified_indices  \
prev_text after_text                                                    
0         1                    -1            -1                   {3}   
2         3                    -1            -1                   {3}   
          4                    -1            -1                   {3}   
          5                    -1            -1                   {3}   
          6                    -1            -1                   {3}   
...                           ...           ...                   ...   
1982      2095                 -1            -1                  {15}   
          2096                 -1            -1                  {15}   
          2097                 -1            -1                  {15}   
          2098                 -1            -1                  {15}   
          2099                 -1            -1                  {15}   

                     to_modified_indices                       changes  
prev_text after_text                                                    
0         1                          {3}      ['replace: [3,4]-[3,4]']  
2         3                          {3}      ['replace: [3,4]-[3,4]']  
          4                          {3}      ['replace: [3,4]-[3,4]']  
          5                          {3}      ['replace: [3,4]-[3,4]']  
          6                          {3}      ['replace: [3,4]-[3,4]']  
...                                  ...                           ...  
1982      2095                      {15}  ['replace: [15,16]-[15,16]']  
          2096                      {15}  ['replace: [15,16]-[15,16]']  
          2097                      {15}  ['replace: [15,16]-[15,16]']  
          2098                      {15}  ['replace: [15,16]-[15,16]']  
          2099                      {15}  ['replace: [15,16]-[15,16]']  

[2080 rows x 6 columns]

In [5]:
edges_df = transformation_log[['prev_text','after_text']]
forward_edges_df = edges_df.set_index('prev_text')
backward_edges_df = edges_df.set_index('after_text')

In [6]:
id_to_text = pd.read_csv('../results/text.csv', index_col="text_id", names = ["text_id", "text"])
text_to_id = pd.read_csv('../results/text.csv', index_col="text", names = ["text_id", "text"])

In [10]:
from nltk.corpus import sentiwordnet as swn
print(swn.senti_synset('breakdown.n.03'))

<breakdown.n.03: PosScore=0.0 NegScore=0.25>


In [37]:
len(neg_list)

4783

In [11]:
list(swn.senti_synsets('slow'))

[SentiSynset('decelerate.v.01'),
 SentiSynset('slow.v.02'),
 SentiSynset('slow.v.03'),
 SentiSynset('slow.a.01'),
 SentiSynset('slow.a.02'),
 SentiSynset('dense.s.04'),
 SentiSynset('slow.a.04'),
 SentiSynset('boring.s.01'),
 SentiSynset('dull.s.08'),
 SentiSynset('slowly.r.01'),
 SentiSynset('behind.r.03')]

In [25]:
happy = swn.senti_synsets('happy', 'a')

In [26]:
list(happy)

[SentiSynset('happy.a.01'),
 SentiSynset('felicitous.s.02'),
 SentiSynset('glad.s.02'),
 SentiSynset('happy.s.04')]

In [27]:
happy = swn.senti_synsets('happy', 'a')
happy0 = list(happy)[1]
happy0.pos_score()

0.75

In [28]:
happy0.neg_score()

0.0

In [29]:
happy0.obj_score()

0.25

## Transformation History

In [1]:
from lineage import InferQuery
from collections import defaultdict
from textattack.shared.utils import words_from_text
queryAPI = InferQuery(dir_pth='./results_a2t_sst2')

2022-07-06 12:47:23.458798: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
api_trace = queryAPI.get_trace_of_output_idx(25)
api_trace

[(2504,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formula romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{21}',
  '{21}'),
 [('replace', (21, 22), (21, 22))],
 (2505,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formulas romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{14}',
  '{14}'),
 [('replace', (14, 15), (14, 15))],
 (2529,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from archaic parody to raunchy sex gags to formulas romantic comedy . ",
  0.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{12}',
  '{12}'),
 [('replace', (12, 13), (12, 13))],
 (2545,
  "nicks , seemingly uncertain what 's going to make

In [3]:
all_traces = queryAPI.get_traces_of_all_outputs()

In [4]:
all_traces[0]

[(2, "it 's a charming and often affecting journey . ", 1.0),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{5}',
  '{5}'),
 [('replace', (5, 6), (5, 6))],
 (28, "it 's a charming and normally affecting journey . ", 1.0)]

In [5]:
len(all_traces)

806

In [6]:
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
import nltk
nltk.download('opinion_lexicon','stopwords')

pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())
stop_words = set(set(stopwords.words('english')))

[nltk_data] Downloading package opinion_lexicon to stopwords...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [7]:
def to_sentiment_category(word):
    if  word in stop_words:
        return 'exclude'
    elif word in pos_list:
        return 'pos'
    elif word in neg_list:
        return 'neg'
    else:
        return 'neutral'

In [24]:
all_edits = {}
for record in all_traces:
    for i,step in enumerate(record):
        if isinstance(step, list):
            for edit in step:
                op = edit[0]
                from_span = edit[1]
                to_span = edit[2]
                from_text = tuple(words_from_text(record[i-2][1])[from_span[0]:from_span[1]])
                to_text = tuple(words_from_text(record[i+1][1])[to_span[0]:to_span[1]])
                
                from_label = record[i-2][2]
                to_label = record[i+1][2]
                
                # to category
                from_cat = to_sentiment_category(from_text[0])
                to_cat = to_sentiment_category(to_text[0])
                
                result_type = record[i+1][2]
                if op not in all_edits:
                    all_edits[op] = defaultdict(dict)
                if to_cat not in all_edits[op][from_cat]:
                    all_edits[op][from_cat][to_cat]={}
                if (from_label, to_label) not in all_edits[op][from_cat][to_cat]:
                    all_edits[op][from_cat][to_cat][(from_label, to_label)] = []
                    
                all_edits[op][from_cat][to_cat][(from_label, to_label)].append((from_text, to_text)) 

In [40]:
edit_freqs = defaultdict(list)
for op in all_edits.keys():
    for from_cat in all_edits[op].keys():
        for to_cat in all_edits[op][from_cat].keys():
            for label_pair in all_edits[op][from_cat][to_cat].keys():
                edit_freqs[label_pair].append((len(all_edits[op][from_cat][to_cat][label_pair]), op, from_cat, to_cat))

In [41]:
for label_pair in edit_freqs.keys():
    print(label_pair)
    edit_freqs[label_pair] = sorted(edit_freqs[label_pair], reverse=True)
    for edit in edit_freqs[label_pair]:
        print(f"\t{edit}")

(1.0, 1.0)
	(659, 'replace', 'neutral', 'neutral')
	(134, 'replace', 'pos', 'neutral')
	(106, 'replace', 'pos', 'pos')
	(55, 'replace', 'neg', 'neg')
	(42, 'replace', 'neg', 'neutral')
	(30, 'replace', 'neutral', 'neg')
	(29, 'replace', 'neutral', 'pos')
	(18, 'replace', 'pos', 'neg')
	(7, 'replace', 'neutral', 'exclude')
	(2, 'replace', 'neg', 'pos')
(1.0, 0.0)
	(52, 'replace', 'neutral', 'neutral')
	(16, 'replace', 'neg', 'neg')
	(10, 'replace', 'pos', 'pos')
	(8, 'replace', 'pos', 'neutral')
	(7, 'replace', 'neutral', 'neg')
	(3, 'replace', 'neg', 'neutral')
	(2, 'replace', 'neutral', 'exclude')
	(1, 'replace', 'pos', 'neg')
(0.0, 0.0)
	(586, 'replace', 'neutral', 'neutral')
	(124, 'replace', 'neg', 'neg')
	(60, 'replace', 'neg', 'neutral')
	(50, 'replace', 'pos', 'pos')
	(36, 'replace', 'neutral', 'pos')
	(32, 'replace', 'pos', 'neutral')
	(18, 'replace', 'neutral', 'neg')
	(13, 'replace', 'neg', 'pos')
	(9, 'replace', 'neutral', 'exclude')
	(1, 'replace', 'pos', 'neg')
(0.0, 1.0)


In [44]:
label_percentage = {}
for label_pair in edit_freqs.keys():
    freq_sum = sum([tup[0] for tup in edit_freqs[label_pair]])
    edits_dict = {}
    for edit in edit_freqs[label_pair]:
        edits_dict[edit[1:]] = edit[0]/freq_sum * 100
    label_percentage[label_pair] = edits_dict
        
for label_pair in label_percentage.keys():
    print(label_pair)
    for edit in sorted(label_percentage[label_pair].keys()):
        print(f"\t{edit}, {label_percentage[label_pair][edit]:.2f}%")

(1.0, 1.0)
	('replace', 'neg', 'neg'), 5.08%
	('replace', 'neg', 'neutral'), 3.88%
	('replace', 'neg', 'pos'), 0.18%
	('replace', 'neutral', 'exclude'), 0.65%
	('replace', 'neutral', 'neg'), 2.77%
	('replace', 'neutral', 'neutral'), 60.91%
	('replace', 'neutral', 'pos'), 2.68%
	('replace', 'pos', 'neg'), 1.66%
	('replace', 'pos', 'neutral'), 12.38%
	('replace', 'pos', 'pos'), 9.80%
(1.0, 0.0)
	('replace', 'neg', 'neg'), 16.16%
	('replace', 'neg', 'neutral'), 3.03%
	('replace', 'neutral', 'exclude'), 2.02%
	('replace', 'neutral', 'neg'), 7.07%
	('replace', 'neutral', 'neutral'), 52.53%
	('replace', 'pos', 'neg'), 1.01%
	('replace', 'pos', 'neutral'), 8.08%
	('replace', 'pos', 'pos'), 10.10%
(0.0, 0.0)
	('replace', 'neg', 'neg'), 13.35%
	('replace', 'neg', 'neutral'), 6.46%
	('replace', 'neg', 'pos'), 1.40%
	('replace', 'neutral', 'exclude'), 0.97%
	('replace', 'neutral', 'neg'), 1.94%
	('replace', 'neutral', 'neutral'), 63.08%
	('replace', 'neutral', 'pos'), 3.88%
	('replace', 'pos', 'n